# 17. 인수를 순회할 때는 방어적으로 하자

In [1]:
def nomalize(numbers):
    total = sum(numbers)
    result = []
    for number in numbers:
        percent = (number/total)*100
        result.append(percent)
    return result

In [4]:
visits = [15, 35, 60]
percentages = nomalize(visits)
print(percentages)

[13.636363636363635, 31.818181818181817, 54.54545454545454]


In [12]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

In [14]:
it = read_visits('./ex_17.txt')
percentages = nomalize(it)
print(percentages)

FileNotFoundError: [Errno 2] No such file or directory: './ex_17.txt'

In [7]:
it = read_visits('./ex_17.txt')
print(list(it))    # 처음 이터레이터를 돌았던 it의 값은 리스트로 변환할 수 있다.
print(list(it))    # 이미 소진함

NameError: name 'read_visits' is not defined

* 이미 소진한 이터레이터를 순회하더라도 오류가 일어나지 않는다.
* for루프와 list생성자, 파이썬 표준 라이브러리의 많은 함수는 정상적인 동작과정에서 StopIteration예외가 일어날 것이라고 기대한다.
* 하지만 이런 함수는 결과가 없는 이터레이터와 결과가 있었지만 이미 소진한 이터레이터의 차이를 알려주지 않는다.



### 이 문제를 해결하기 위해선 입력 이터레이터를 명시적으로 소진하고 전체 콘텐츠 복사본을 리스트에 저장해야 한다.
> 입력 이터레이터를 방어적으로 복사하는 함수이다.

In [8]:
def nomalize_copy(numbers):
    numbers = list(numbers)     # 이터레이터를 복사함
    total = sum(numbers)
    result = []
    for number in numbers:
        percent = (number/total)*100
        result.append(percent)
    return result

In [9]:
# 올바르게 동작함

it = read_visits('./my_17.txt')
percentages = nomalize_copy(it)
print(percentages)

NameError: name 'read_visits' is not defined

* 위 방법의 문제는 이터레이터의 복사본이 크면 메모리 고갈 문제가 발생한다는 점이다.
* 해결방법 : 호출될 때마다 새 이터레이터를 반환하는 함수 만들기

In [15]:
def nomalize_func(get_iter):
    total = sum(get_iter())             # 새 이터레이터
    result = []
    for number in get_iter():           # 새 이터레이터
        percent = (number/total)*100
        result.append(percent)
    return result

In [17]:
# nomalize_func을 사용하려면 제너레이터를 호출해서 매번 새 이터레이터를 생성하는 람다표현식을 넘겨줘야 한다.
# 이런 방법은 세련되지 못하다.
percentages = nomalize_func(lambda : read_visits(path))

NameError: name 'path' is not defined

#### 더 좋은 방법은 이터레이터 프로토콜(iterater protocol)을 구현한 새 컨테이너 클래스를 제공하는 것
* 이터레이터 프로토콜 : 파이썬의 for 루프와 관련 표현식이 컨테이너 타입의 콘텐츠를 탐색하는 방법을 나타낸다.

python 에서는
1. for x in foo 같은 문장을 만나면 실제로 iter(foo)를 호출
2. 내장함수 iter는 특별 메서트인 foo.__iter__를 호출
3. __iter__메서드는(__next__라는 특별한 메서드를 구현한다) 이터레이터 객체를 반환해야 함.
4. 마지막으로 for루프는 이터레이터를 모두 소진할 때까지 (StopIteration예외가 발생할때 까지) 이터레이터 객체에 내장함수 next를 계속 호출

##### 쉽게 말해 클레스의  __iter__ 메서드를 제너레이터로 구현하면 이렇게 동작하게 만들 수 있다.

In [18]:
# 데이터를 담은 파일을 읽는 이터러블(순회가능) 컨테이너 클래스

class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [19]:
# 이렇게 정의한 컨테이너 타입은 원래의 함수에 수정을 가하지 않고 넘겨도 제대로 동작
visit = ReadVisits(path)
percentages = nomalize(visits)
print(percentages)

#### 제대로 동작하는 이유
* nomalize의 sum 메서드가 새 이터레이터 객체를 할당하려고 ReadVisits.\__iter__를 호출
* 숫자 정규화하는 for루프도 두 번째 이터레이터 객체를 할당할때 \__iter__를 호출
* 두 이터레이터가 독립적으로 동작하므로 각각 입력데이터 값을 얻을 수 있다.
* 단점은 입력데이터를 여러번 읽는다

#### 한발짝 더 나아가기
> 파라미터가 단순한 이터레이터가 아님을 보장하는 함수를 작성

In [1]:
def nomalize_defensive(numbers):
    if iter(numbers) is iter(numbers):
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for number in numbers:
        percent = (number/total)*100
        result.append(percent)
    return result

* nomalize_defensive는 nomalize_copy처럼 입력 이터레이터 전체를 복사하고 싶지는 않지만, 입력데이터를 여러번 순회해야 할 때 좋다.
* 이 함수는 list와 ReadVisits를 입력으로 받으면 입력이 컨테이너이므로 기대한 대로 동작
* 이터레이터 프로토콜을 따르는 어떤 컨테이너 타입에 대해서도 제대로 동작

In [ ]:
visits = [15, 30, 65]
nomalize_defensive(visits) # 오류 없음
visits = ReadVisits(path)
nomalize_defensive(visits) # 오류 없음

컨테이너가 아닌 이터레이터 입력에서는 오류를 일으킨다.

In [ ]:
it = iter(visits)
nomalize_defensive(it)

## 핵심정리
* 입력 인수를 여러 번 순회하는 함수를 작성할때 주의하자. 입력 인수가 이터레이터라면 이상하게 동작해서 값을 잃어버릴 수 있다.
* 파이썬의 이터레이터 프로토콜은 컨테이너와 이터레이터가 내장함수 iter, next와 for루프 및 관련 표현식과 상호 작용하는 방법을 정의한다.
* __iter__메서드를 제너레이터로 구현하면 자신만의 이터러블 컨테이너 타입을 쉽게 정의할 수 있다.
* 어떤 값에 iter를 두 번 호출했을 때 같은 결과가 나오고 내장 함수 next로 전진시킬 수 있다면 그 값은 컨테이너가 아닌 이터레이터다.